## Fitting Data

Leggi il paper sul fitting di Hogg!

Assume the noise/scatter in our measurements (the residuals) is generated by a Gaussian process, i.e.:

$$ y_i = a x_i + b + r_i $$

Power laws in log become linear -> linearize the problem -> fit a line to data


$$ p(y_i|x_i, M(a, b), \sigma) = N(y_i - M(x)|\sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left( - \frac{(y_i - M(x_i))^2}{2 \sigma^2} \right). $$

M è il modello ^
Faccio i minimi quadrati perché la maximum likelihood ha tutte le proprietà che mi piacciono (?)
Immagine M-sigma relation: vedo diverse popolazioni. Che faccio con questi data points?
Vedo kind of linear trend con degli outliers: non posso buttarli via e basta!


Iniziamo fittando solo i punti neri:
*scipy.optimize.fmin* trova il minimo di una funzione ( che gli dai in pasto )
Se metto i punti rossi in ballo, auguri! 

#### How to deal with outliers

Abbiamo già visto che la media è suscettibile agli outliers, quindi ci sarà un discorso simile in questo caso.

In generale, nel fare data analysis non c'è un modo giusto o sbagliato di fare le cose! Ultimately, vuoi una buona soluzione che è buona a prevedere la prossima osservazione.

Ok, che succede se cambiamo la *loss*? "Gli errori sono stati sottostimati": cambio la likelyhood
Minimizzo il chiquadro se il processo è gaussiano: qui non è gaussiano, quindi..?

*Huber Loss Function*: prende il chiquadro e aggiunge correzione lineare. I punti "lontani" dal fit vengono essenzialmente downgraded. Bene. proviamo a rifare il fit con gli outliers ma usando questa funzione al posto del chiquadro? ( se ho ben capito )

Parola magica: *hyperparameters*.

## Goodness of fit & Model Comparison

Voglio capire quanto è buono il mio fit.

Modello = crap => quel che trovi = crap

Quindi: cerchiamo di formalizzare cosa intendiamo con "good" e "bad".

Randomly draw points from the fit: se *evaluate* la likelyhood con quei punti, mi aspetto che siano *vicini* al picco della mia likelihood!

Nel caso gaussiano è facile: ln(L) è distribuito come il chiquadro e bbona.

***The mean of the $\chi^2$ distribution is $N-k$ and its standard deviation is $\sqrt{2(N-k)}$.***

We define the $\chi^2$ per degree of freedom, $\chi^2_\mathrm{dof}$, as

$$\chi^2_\mathrm{dof} = \frac{1}{N-k}\sum_{i=1}^N z^2_i.$$

where $k$ is the number of model parameters determined from the data.

- For a good fit, we would expect that $\chi^2_\mathrm{dof}\approx 1$. 
- If $\chi^2_\mathrm{dof}$ is significantly larger than 1, or $(\chi^2_\mathrm{dof}-1)>> \sqrt{2/(N-k)}$, then it is likely that we are not using the correct model.
- If data uncertainties are **(over)under-estimated** then this can lead to improbably **(low)high $\chi^2_\mathrm{dof}$**, as seen below.  

![Ivezic, Figure 4.1](http://www.astroml.org/_images/fig_chi2_eval_1.png)

## Model Comparison

Il chiquadro coi gradi di libertà è buono quando usi i minimi quadrati e basta, praticamente.
Come faccio i minimi quadrati con la Huber Loss Function? Eh.

> In generale, mi aspetto che il modello con la massima likelihood sia il migliore

Occam's Razor: "all else being equal, the less complex is favored". In statistica, lo implemento penalizzando i modelli con un numero maggiore di parametri.

Standard Approach in Frequentist: Akaike Information Criterion (AIC)

$$ \mathrm{AIC}_M \equiv -2\ln[L^0(M)] + 2k + \frac{2k(k+1)}{N-k-1}, $$

where $k$ is the number of model parameters and $N$ is the number of data points.

- For a Gaussian distribution, the first term is equal to $\chi^2$.
- **The model with lowest AIC is the most favored.**
- If all models are equally successful at fitting the data (equal $L^0$ values) then the second and third terms penalize model complexity such that the model with fewest free parameters wins. 

The AIC is a good quick and dirty estimate for the goodness of fit based on some asymptotic approximations (cf.  [Wasserman's textbook](https://link.springer.com/book/10.1007/978-0-387-21736-9) if you're interested. The preferred approach is **cross validation**. Cross validation is at the heart of all deep-learning workflows and will be explored later in the course.

> In altre parole, cerchi un equilibrio tra aggiungere cose per migliorare il modello e tra il non esagerare nell'aggiungere complessità

Il preferred approach is *cross validation*.

## Confidence Estimating: Bootstrap & Jackknife

Two model techniques per estimate confidence intervals. Reso possibile dal pc, resampling strategies, e ottimo in analisi frequentista.

##### Bootstrap Method

In bootstrapping, we map the uncertainty of model parameters by re-sampling from our data (with replacement) $B$ times. This is relatively new (Efron, 1979) and was named in inspiration of "*pulling oneself up by one's bootstraps*".

When sampling from observed data of length $N$ with replacement, there are $N!$ distinct combinations of new observed datasets, and there is only a $N!/N^N$ probability of getting back the original dataset (even for $N=10$ this is only $0.00036$). 

With $B$ new bootstrap datasets, we compute our statistics on each to obtain $B$ measures of our parameters. So, if we have $i=1,\dots,N$ data points in $\{x_i\}$, we draw $N$ of them to make a new sample, where some values of $\{x_i\}$ will be used more than once (and this is ok).

**EXAMPLE**
- We have an original dataset of $1000$ points drawn from a Gaussian distribution. 
- We can only measure the standard deviation of the distribution once with this dataset. 
- Our previous strategies showed that we can use a *Fisher estimate* or an *analytic estimate* of the sample standard deviation uncertainty. 
- Instead, we use the bootstrap method to resample the data $10,000$ times, and compute the standard deviation and $\sigma_G$ on each new dataset. 
- This will map out the uncertainty distribution of those statistics, allowing us to quote confidence limits in our actual measured value. 

##### Jackknife Method 

This is similar to bootsrapping except that we don't use a sample size of $N$. Rather, ***we leave off one or more of the observations*** from $\{x_i\}$. As with bootstrap, we do this multiple times, generating samples from which we can determine our uncertainties.

- If we leave out just one datapoint each time, we can make $N$ such datasets that each contain $(N-1)$ data points.
- We compute our statistic or find our model parameters in each of these jackknife datasets.

For jackknifing by leaving out one datapoint, the bias-corrected jackknife estimate of a statistic $\alpha$ is

$$ \alpha^J = \alpha_N + \Delta\alpha,$$

where $\alpha_N$ is the statistic computed on the original dataset, and 

$$ \Delta\alpha = (N-1)\left( \alpha_N - \frac{1}{N}\sum_{i=1}^N \alpha^*_i \right)$$

where $\{\alpha_i^*\}$ are the statistics computed on the $N$ jackknife datasets. 

For asymptoticically normal estimators, the standard error on $\alpha^J$ is

$$ \sigma_\alpha = \sqrt{\frac{1}{N(N-1)}\sum_{i=1}^N [N\alpha_N - \alpha^J - (N-1)\alpha^*_i]^2}.$$

Confidence limits on $\alpha$ can be computed using the Student's $t$ distribution with $t=(\alpha-\alpha^J)/\sigma_\alpha$ and $(N-1)$ degrees of freedom.

> *ATTENZIONE*: Il jackknife è pessimo se hai il ranking dei dati: se strappi via un dato e non lo sostituisci, capisci che la mediana va al diavolo pesantemente! For things that do NOT rely on ranking, jackknife is better than bootstrap though!

**NOTES**
- The jackknife standard error is more reliable than the bias correction.
- Standard jackknife (removing one point) does well for statistics like the mean and standard deviation, but is poor with rank-based statistics (e.g., median, quantiles, $\sigma_G$). 
- This is because (as we've seen) rank-based statistics are not very sensitive to adding/removing single data points. So jackknifing can give many identical values of the statistic!
- This can be resolved by modifying the jackknife to leave off more than one datapoint.


**EXAMPLE**
- We compute jackknife uncertainty estimates for the width of a Gaussian distribution, using the same data as in the bootstrap example.
- The $N$ jackknife estimates of $\sigma$ and $\sigma_G$ will be shown, but the `astroML` jackknife method automatically incorporates the bias correction mentioned above.


### When to bootstrap or jackknife?

- Jackknife estimates are usually easier to calculate, easier to apply for complex sampling schemes, and automtically remove bias.
- Bootstrap is better for computing confidence intervals because it maps out the full distribution of the statistic instead of assuming asymptotic normality.
- Bootstrap is random resampling so gives different results each time. Whereas jackknifing gives repeatable results.

It is generally a good idea to use both methods and compare the results. Use either/both with caution with $N$ is small!

But in general, **cross validation** is the way to go. We'll see this carefully in a few lectures.
